In [ ]:
#Create Peoject object by using project id and project token. 
#Click on 'More' (Triple dots) icon in Notebook toolbar. Then click 'Insert Project Token' to generate the necessary code here.

In [ ]:
# Click the 'Find and Add Data' (0100) icon in Notebook toolbar. 
#Click on 'source_sentences_shaped.csv' file and select 'pandas data frame' from drop down
# In cae you are using this notebook in your local environment then you can directly read the data from the csv file using Pandas"

In [ ]:
source_sentences = list(df_data_1['text'])
len(source_sentences)

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel 
import torch 
import torch.nn.functional as F
import numpy as anp
from sentence_transformers import SentenceTransformer

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

In [ ]:
# encoding the source sentences
encoded_input = tokenizer(source_sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# computing the embeddings
with torch.no_grad():
    embedding_model = model(**encoded_input)


In [ ]:
# Perform pooling 
sentence_embeddings = mean_pooling(embedding_model, encoded_input['attention_mask'])
 # Normalise embeddings 
sentence_embeddings = F.normalize(sentence_embeddings, p = 2, dim = 1)
sentence_embeddings = sentence_embeddings.detach().numpy().flatten()
print("Embeddings shape: {}".format(sentence_embeddings.shape))

In [ ]:
import numpy as np
sentence_embeddings_hist = np.histogram(sentence_embeddings, density = True)
train_embeddings_vals = sentence_embeddings_hist[0].tolist()
train_embeddings_bins = sentence_embeddings_hist[1].tolist()[:-1]
train_embeddings = {'train_vals': train_embeddings_vals, 'train_bins': train_embeddings_bins}

In [ ]:
import pandas as pd
embeddings_df = pd.DataFrame(train_embeddings)

In [ ]:
project.save_data("training_data_drift_stats.csv", embeddings_df.to_csv(index=False))

In [ ]:
#In case you are uing this notebook in your local machine/laptop use following code to save the file"
#embeddings_df.to_csv('train_data_drift_stats.csv', index=False, mode='w+')